# DATA CLEANING

## Importation

In [38]:
import pandas as pd

#Import data
df = pd.read_csv('client.csv')
df.head(3)

,clIent number,Attrition Flag,Customer-age,Gender,Education LEVEL,marital status,Income_Category,MonthsBook,CD,Avg_Utilization_Ratio,PhoneNumber,Address,dependents
0,768805383,Existing Customer,45.0,M,High School,Married,$60K - $80K,39,NaN,0.061,+33--6-23-45-67-89,"Berlin, Rue de Rivoli, 10117",2.0
1,818770008,Existing Customer,49.0,F,Graduate,Single,Less than $40K,44,8256.0,0.105,+44 7911 --123456,"Berlin, Rue de Rivoli, 00187",NaN
2,713982108,Existing Customer,51.0,M,Graduate,Married,$80K - $120K,36,NaN,0.000,+4923 456 7890,"Madrid, Via Condotti, 75001",NaN


- Renaming columns 

In [ ]:
import re

# Function to clean column names
def fix_col(col):  
    return (
        re.sub(r'\d+\.\s+', '', col)  
        .lower()  
        .replace(' ', '_')
        .replace('-', '_')  
    )

df.rename(columns=fix_col, inplace=True)

In [40]:
# fix other columns 
df.rename(columns={
    'monthsbook' : 'months_book',
    'cd' : 'credit_card_balance',
    'phonenumber' : 'phone_number'
    }, inplace=True)

df.head(1)

,client_number,attrition_flag,customer_age,gender,education_level,marital_status,income_category,months_book,credit_card_balance,avg_utilization_ratio,phone_number,address,dependents
0,768805383,Existing Customer,45.0,M,High School,Married,$60K - $80K,39,NaN,0.061,+33--6-23-45-67-89,"Berlin, Rue de Rivoli, 10117",2.0


## Missing Values

In [41]:
# Info about te data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   client_number          328 non-null    object 
 1   attrition_flag         324 non-null    object 
 2   customer_age           290 non-null    float64
 3   gender                 316 non-null    object 
 4   education_level        330 non-null    object 
 5   marital_status         330 non-null    object 
 6   income_category        303 non-null    object 
 7   months_book            330 non-null    int64  
 8   credit_card_balance    294 non-null    float64
 9   avg_utilization_ratio  330 non-null    float64
 10  phone_number           330 non-null    object 
 11  address                330 non-null    object 
 12  dependents             73 non-null     float64
dtypes: float64(4), int64(1), object(8)
memory usage: 33.6+ KB


In [42]:
# Calculate percentage of missing values before fixing them
df.isnull().mean() *100

client_number             0.606061
attrition_flag            1.818182
customer_age             12.121212
gender                    4.242424
education_level           0.000000
marital_status            0.000000
income_category           8.181818
months_book               0.000000
credit_card_balance      10.909091
avg_utilization_ratio     0.000000
phone_number              0.000000
address                   0.000000
dependents               77.878788
dtype: float64

In [43]:
# We see that the dependets column is 77% Null , so it's better to drop it 
df = df.drop(columns = 'dependents')


- Replacing empty strings with Null

In [ ]:
def empty_string_count(df):
    # Regular expression to identify empty strings or strings with only spaces
    empty_string_pattern = r'^\s*$'
    
    # Count the number of empty strings in each column 
    empty_string_count = df.apply(
        lambda col: col.str.match(empty_string_pattern).sum() if col.dtype == 'object' else 0
    )

    # Print the count of empty strings per column
    print(empty_string_count)

# Display the count of empty strings before cleaning
print('Before Replacing:')
empty_string_count(df)

# Replace empty strings with pd.NA
df = df.replace(r'^\s*$', pd.NA, regex=True)

# Display the count of empty strings after cleaning
print('\nAfter Replacing:')
empty_string_count(df)

Before Replacing:
client_number            1
attrition_flag           6
customer_age             0
gender                   3
education_level          3
marital_status           0
income_category          0
months_book              0
credit_card_balance      0
avg_utilization_ratio    0
phone_number             0
address                  0
dtype: int64

After Replacing:
client_number            0
attrition_flag           0
customer_age             0
gender                   0
education_level          0
marital_status           0
income_category          0
months_book              0
credit_card_balance      0
avg_utilization_ratio    0
phone_number             0
address                  0
dtype: int64


- Dropping rows with missing critical data


In [45]:
# Drop rows where 'client_number' or 'attrition_flag' have null values, as they are mandatory
df = df.dropna(subset=['client_number', 'attrition_flag'])

# Check if there are any missing values left in the 'client_number' and 'attrition_flag' columns
missing_client_number = df['client_number'].isnull().sum()
missing_attrition_flag = df['attrition_flag'].isnull().sum()

# Print the result
print(f"Missing 'client_number' values: {missing_client_number}")
print(f"Missing 'attrition_flag' values: {missing_attrition_flag}")

Missing 'client_number' values: 0
Missing 'attrition_flag' values: 0


### Filling Missing values 

In [ ]:
# Filling with 'unknown' in education_level
df['education_level'].fillna('unknown', inplace=True)

In [47]:
# Filling age with the mean value based on marital status
mean_values = df.groupby('marital_status')['customer_age'].mean()

# Fill missing customer_age values with the mean for the corresponding marital status
df['customer_age'] = df.apply(
    lambda row: mean_values[row['marital_status']] if pd.isnull(row['customer_age']) else row['customer_age'],
    axis=1
)


In [ ]:
# Filling credit_card_balance using Iterative imputer (regression) 
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer()

# Fit and transform  
df['credit_card_balance'] = iterative_imputer.fit_transform(df[['credit_card_balance']])


In [ ]:
# Filling gender :
# since it is not numercial , we can't apply knn or any other imputer , so we fill it using next line
df['gender'].fillna(method='bfill', inplace=True)

# If we want to fill using the previous line ,we do : methode = 'ffill

In [ ]:
# Filling income_category with the most common value
most_common = df['income_category'].mode()[0]  # Selecting the most common value
df['income_category'].fillna(most_common, inplace=True)

- Check if all cleaned 

In [51]:
df.isnull().sum()

client_number            0
attrition_flag           0
customer_age             0
gender                   0
education_level          0
marital_status           0
income_category          0
months_book              0
credit_card_balance      0
avg_utilization_ratio    0
phone_number             0
address                  0
dtype: int64

In [52]:
# Get rid of duplicates :
df = df.drop_duplicates()

## Bad Values

Out of scale, misspelled, inconsistent formats, etc ...

### Fixing Categorical 

In [53]:
# Get a descriptive summary of categorical features
df.describe(include='object')

,client_number,attrition_flag,gender,education_level,marital_status,income_category,phone_number,address
count,316,316,316,316,316,316,316,316
unique,316,2,13,8,4,6,10,112
top,710558058,Existing Customer,M,Graduate,Married,Less than $40K,+44 20 7946 0958,"Rome, Gran Via, NW1 6XE"
freq,1,286,138,100,180,93,47,10


- Fixing gender 

In [ ]:
# Show statistics for each unique value of the 'gender' feature
df.groupby('gender').describe()

customer_age                                                            \
              count       mean         std   min    25%        50%        75%   
gender                                                                          
-M              3.0  45.666667    2.886751  44.0  44.00  44.000000  46.500000   
F              88.0  49.904978   12.779179 -51.0  46.00  51.000000  56.161859   
F              14.0  40.449634   25.498784 -45.0  40.25  45.500000  48.750000   
Fe             18.0  43.330100   25.475105 -52.0  41.50  47.285714  54.500000   
Femme           3.0  40.000000   31.192948   4.0  30.50  57.000000  58.000000   
M             138.0  59.956327   63.641299 -51.0  46.25  51.000000  55.842593   
M              18.0  47.091524    8.788101  24.0  43.50  47.500000  52.250000   
Male            2.0  43.500000    0.707107  43.0  43.25  43.500000  43.750000   
f              17.0  53.871835    5.796521  44.0  48.00  56.000000  57.000000   
femme           1.0  70.000000         NaN  70.0  70.00  70.000000  70.000000   
femme           1.0  46.000000         NaN  46.0  46.00  46.000000  46.000000   
m              12.0  84.553953  116.744663  43.0  47.75  50.500000  56.161859   
m               1.0  57.000000         NaN  57.0  57.00  57.000000  57.000000   

              months_book             ... credit_card_balance                \
          max       count       mean  ...                 75%           max   
gender                                ...                                     
-M       49.0         3.0  33.333333  ...        29414.000000  34516.000000   
F        67.0        88.0  38.840909  ...        10108.250000  32643.000000   
F        60.0        14.0  32.214286  ...        18814.750000  34516.000000   
Fe       73.0        18.0  37.277778  ...        23460.250000  34516.000000   
Femme    59.0         3.0  35.000000  ...         8393.941637  10452.883274   
M       534.0       138.0  39.463768  ...        14965.750000  34516.000000   
M        66.0        18.0  37.055556  ...        10452.883274  23218.000000   
Male     44.0         2.0  25.500000  ...        21404.000000  27259.000000   
f        62.0        17.0  41.411765  ...        11376.000000  34516.000000   
femme    70.0         1.0  56.000000  ...         3252.000000   3252.000000   
femme    46.0         1.0  40.000000  ...        19458.000000  19458.000000   
m       455.0        12.0  39.250000  ...        13561.000000  24407.000000   
m        57.0         1.0  36.000000  ...        19482.000000  19482.000000   

       avg_utilization_ratio                                              \
                       count      mean       std    min      25%     50%   
gender                                                                     
-M                       3.0  0.245000  0.358121  0.000  0.03950  0.0790   
F                       88.0  0.363114  0.282090  0.000  0.12750  0.3360   
F                       14.0  0.241929  0.304626  0.000  0.01225  0.1555   
Fe                      18.0  0.155167  0.173652  0.000  0.04500  0.0905   
Femme                    3.0  0.152333  0.141260  0.000  0.08900  0.1780   
M                      138.0  0.219536  0.225519  0.000  0.05825  0.1375   
M                       18.0  0.305778  0.319957  0.000  0.06225  0.1265   
Male                     2.0  0.299000  0.422850  0.000  0.14950  0.2990   
f                       17.0  0.234941  0.246262  0.000  0.06000  0.1310   
femme                    1.0  0.460000       NaN  0.460  0.46000  0.4600   
femme                    1.0  0.074000       NaN  0.074  0.07400  0.0740   
m                       12.0  0.273083  0.266272  0.000  0.12200  0.1570   
m                        1.0  0.055000       NaN  0.055  0.05500  0.0550   

                        
            75%    max  
gender                  
-M      0.36750  0.656  
F       0.61050  0.877  
F       0.24400  0.963  
Fe      0.16875  0.591  
Femme   0.22850  0.279  
M       0.28650  0.987  
M  

In [55]:
# Strip any leading/trailing spaces
df["gender"] = df["gender"].str.strip()

# Strip the specific character "-" (if present)
df["gender"] = df["gender"].str.replace("-", "", regex=False)

# Convert the column to lowercase
df["gender"] = df["gender"].str.lower()

# Replace instances starting with 'f' or 'F' with a single 'f'
df["gender"] = df["gender"].str.replace(r'^[Ff].*', 'f', regex=True)

# Replace instances starting with 'm' or 'M' with a single 'm'
df["gender"] = df["gender"].str.replace(r'^[mM].*', 'm', regex=True)

# Group by 'gender' and describe
df.groupby('gender').describe()


customer_age                                                            \
              count       mean        std   min   25%   50%        75%    max   
gender                                                                          
f             142.0  48.518986  16.612895 -52.0  45.0  51.0  56.647436   73.0   
m             174.0  59.869356  64.401792 -51.0  45.0  50.0  55.370370  534.0   

       months_book             ... credit_card_balance           \
             count       mean  ...                 75%      max   
gender                         ...                                
f            142.0  38.345070  ...        10452.883274  34516.0   
m            174.0  38.913793  ...        14968.750000  34516.0   

       avg_utilization_ratio                                            \
                       count      mean       std  min      25%     50%   
gender                                                                   
f                      142.0  0.303655  0.274698  0.0  0.06925  0.2130   
m                      174.0  0.232557  0.241666  0.0  0.05950  0.1405   

                        
            75%    max  
gender                  
f       0.50625  0.963  
m       0.31700  0.987  

[2 rows x 32 columns]

- Fixing education_level

In [ ]:
# Show statistics for each unique value of the education_level feature
df.groupby('education_level').describe()

customer_age                                              \
                       count       mean         std       min        25%   
education_level                                                            
College                 27.0  50.024639   10.567005   6.00000  46.500000   
Doctorate               15.0  52.121413    5.873290  44.00000  48.000000   
Graduate               100.0  54.196788   42.265825 -51.00000  45.750000   
High School             67.0  52.330349   22.650136  24.00000  45.000000   
Post-Graduate           14.0  79.638736  115.371373  39.00000  44.750000   
Uneducated              51.0  62.567845   82.808829 -51.00000  44.500000   
Unknown                 39.0  45.506287   23.091535 -52.00000  44.000000   
unknown                  3.0  56.005935    0.638553  55.37037  55.685185   

                                             months_book             ...  \
                  50%        75%         max       count       mean  ...   
education_level                                                      ...   
College          53.0  56.323718   60.000000        27.0  40.629630  ...   
Doctorate        50.0  55.823718   66.000000        15.0  40.066667  ...   
Graduate         51.0  56.647436  455.000000       100.0  38.900000  ...   
High School      49.0  55.185185  220.000000        67.0  38.880597  ...   
Post-Graduate    49.5  55.735577  480.000000        14.0  35.571429  ...   
Uneducated       49.0  56.000000  534.000000        51.0  36.509804  ...   
Unknown          50.0  54.500000   65.000000        39.0  39.871795  ...   
unknown          56.0  56.323718   56.647436         3.0  36.000000  ...   

                credit_card_balance          avg_utilization_ratio            \
                                75%      max                 count      mean   
education_level                                                                
College                19005.500000  34516.0                  27.0  0.204704   
Doctorate              13439.941637  24159.0                  15.0  0.337733   
Graduate               13188.750000  34516.0                 100.0  0.283610   
High School            12888.000000  34516.0                  67.0  0.227299   
Post-Graduate          20059.750000  34516.0                  14.0  0.161429   
Uneducated             10779.941637  34516.0                  51.0  0.303176   
Unknown                11334.941637  34516.0                  39.0  0.277897   
unknown                21547.941637  32643.0                   3.0  0.280333   

                                                                  
                      std   min      25%     50%      75%    max  
education_level                                                   
College          0.206851  0.00  0.05400  0.1330  0.29250  0.760  
Doctorate        0.294701  0.05  0.07750  0.2460  0.62650  0.793  
Graduate         0.265363  0.00  0.06675  0.1905  0.50825  0.877  
High School      0.234841  0.00  0.06150  0.1460  0.32300  0.788  
Post-Graduate    0.170174  0.00  0.03875  0.0900  0.25300  0.463  
Uneducated       0.269769  0.00  0.09400  0.2240  0.47750  0.987  
Unknown          0.300982  0.00  0.05250  0.1270  0.49750  0.963  
unknown          0.318073  0.00  0.10750  0.2150  0.42050  0.626  

[8 rows x 32 columns]

In [57]:
df["education_level"] = df["education_level"].str.replace("unknown", "Unknown", regex=False)

- Fixing phone_number

In [58]:
df["phone_number"] = df["phone_number"].str.strip()
df["phone_number"] = df["phone_number"].str.replace("-", '') 
df["phone_number"] = df["phone_number"].str.replace(" ", '') 
df["phone_number"] = df["phone_number"].apply(lambda x: x[:-9] + ' ' +x[-9:-8] + ' ' + x[-8:-6] + ' ' + x[-6:-4] + ' ' + x[-4:-2] + ' ' + x[-2:] )
df['phone_number'].head(3)

0     +33 6 23 45 67 89
1    +447 9 11 12 34 56
2     +49 2 34 56 78 90
Name: phone_number, dtype: object

- Fixing address

In [59]:
# 5 - Cleaning addresses , splitting them
df[['City', 'Street', 'Postal Code']] = df['address'].str.split(',', expand=True)
df.head(3)

,client_number,attrition_flag,customer_age,gender,education_level,marital_status,income_category,months_book,credit_card_balance,avg_utilization_ratio,phone_number,address,City,Street,Postal Code
0,768805383,Existing Customer,45.0,m,High School,Married,$60K - $80K,39,10452.883274,0.061,+33 6 23 45 67 89,"Berlin, Rue de Rivoli, 10117",Berlin,Rue de Rivoli,10117
1,818770008,Existing Customer,49.0,f,Graduate,Single,Less than $40K,44,8256.000000,0.105,+447 9 11 12 34 56,"Berlin, Rue de Rivoli, 00187",Berlin,Rue de Rivoli,00187
2,713982108,Existing Customer,51.0,m,Graduate,Married,$80K - $120K,36,10452.883274,0.000,+49 2 34 56 78 90,"Madrid, Via Condotti, 75001",Madrid,Via Condotti,75001


### Fixing Continuous

In [60]:
# Get a descriptive summary of Continuous features
df.describe()


,customer_age,months_book,credit_card_balance,avg_utilization_ratio
count,316.000000,316.000000,316.000000,316.000000
mean,54.768873,38.658228,10452.883274,0.264506
std,49.329499,7.350864,9268.733426,0.259049
min,-52.000000,20.000000,1438.300000,0.000000
25%,45.000000,35.000000,3207.750000,0.061750
50%,50.000000,36.000000,7953.500000,0.163000
75%,56.000000,43.000000,13293.250000,0.447000
max,534.000000,56.000000,34516.000000,0.987000


In [61]:
# get rid of outliers using Z score
for column in ['customer_age', 'months_book', 'credit_card_balance', 'avg_utilization_ratio']:
    # Calculate the Z-score for the current column
    z_score = (df[column] - df[column].mean()) / df[column].std()
    
    # Identify the outliers (Z-score > 2 or < -2)
    bad_outliers = df[z_score.abs() > 3]
    
    # Print or check the outliers (Optional)
    print(f"Outliers in {column}:")
    print(bad_outliers)
    
    # Remove the outliers from the original dataframe
    df = df.loc[z_score.abs() <= 2]

df.describe(include='all')

Outliers in customer_age:
    client_number     attrition_flag  customer_age gender education_level  \
8       710930508  Existing Customer         370.0      m      Uneducated   
16      709967358  Existing Customer         480.0      m   Post-Graduate   
19      709327383  Existing Customer         455.0      m        Graduate   
31      712991808  Existing Customer         534.0      m      Uneducated   
122     714877233  Existing Customer         220.0      m     High School   

    marital_status income_category  months_book  credit_card_balance  \
8           Single     $60K - $80K           36              22352.0   
16          Single    $80K - $120K           36              30367.0   
19         Married  Less than $40K           37              14470.0   
31         Married     $60K - $80K           48               2451.0   
122        Married    $80K - $120K           47              12262.0   

     avg_utilization_ratio         phone_number  \
8                    0.113 

,client_number,attrition_flag,customer_age,gender,education_level,marital_status,income_category,months_book,credit_card_balance,avg_utilization_ratio,phone_number,address,City,Street,Postal Code
count,252,252,252.000000,252,252,252,252,252.000000,252.000000,252.000000,252,252,252,252,252
unique,252,2,NaN,2,7,4,6,NaN,NaN,NaN,7,105,5,5,5
top,768805383,Existing Customer,NaN,m,Graduate,Married,Less than $40K,NaN,NaN,NaN,+4915 2 34 56 78 90,"Rome, Gran Via, NW1 6XE",Rome,Unter den Linden,75001
freq,1,228,NaN,141,77,139,70,NaN,NaN,NaN,57,7,57,70,59
mean,NaN,NaN,49.915283,NaN,NaN,NaN,NaN,38.214286,8981.963749,0.241710,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,7.031273,NaN,NaN,NaN,NaN,6.147726,6552.247681,0.222519,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,24.000000,1438.300000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,45.000000,NaN,NaN,NaN,NaN,35.000000,3374.000000,0.069750,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,50.000000,NaN,NaN,NaN,NaN,36.000000,7948.500000,0.166000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,55.370370,NaN,NaN,NaN,NaN,43.000000,11488.000000,0.367500,NaN,NaN,NaN,NaN,NaN


## Data Validation using Schemas
- Ensures that data conforms to specific types and constraints, reducing the risk of errors during processing or analysis.
- Helps catch incorrect, out-of-range values, missing data, and other issues before they cause problems.
- Acts as documentation, specifying what the data should look like, which is useful for anyone working with or interpreting the data. 

In [62]:
import pandera as pa

df['customer_age'] = df['customer_age'].astype(int)

schema = pa.DataFrameSchema({
    'client_number': pa.Column(pa.String, unique=True),
    'attrition_flag': pa.Column(pa.String, checks=pa.Check.isin(['Existing Customer', 'Attrited Customer'])),
    'customer_age': pa.Column(pa.Int, checks=pa.Check.in_range(5, 120)),
    'gender': pa.Column(pa.String, checks=pa.Check.isin(['m', 'f'])),
    'education_level': pa.Column(pa.String),
    'marital_status': pa.Column(pa.String),
    'income_category': pa.Column(pa.String),
    'months_book': pa.Column(pa.Int, checks=pa.Check.ge(0)),
    'credit_card_balance': pa.Column(pa.Float, checks=pa.Check.ge(0)),
    'avg_utilization_ratio': pa.Column(pa.Float, checks=pa.Check.in_range(0, 1)),
    'phone_number': pa.Column(pa.String),
    'address': pa.Column(pa.String),
    'City': pa.Column(pa.String),
    'Street': pa.Column(pa.String),
    'Postal Code': pa.Column(pa.String)
})

validated_df = schema.validate(df)